---
# Data Scraping
---

In [1]:
# Importing tools

import requests
# import urllib.request
import time
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)



---
### 1. Fetch the content by URL
---

##### The data was collected from https://co.vid19.sg/singapore/dashboard. No API or loop was needed to accumulate and collect the data.

In [2]:
# Code for parsing HTML Tables in Python with BeautifulSoup and pandas
# ref https://srome.github.io/Parsing-HTML-Tables-in-Python-with-BeautifulSoup-and-pandas/?fbclid=IwAR3R12X84U6xkgZ0_a02gPPbKm-Tk6MnYl6ga7R7srPDNXXkzfMU79jhZr4

url = "https://co.vid19.sg/singapore/cases/search.html"
response = requests.get(url)
response.text[:100] # Access the HTML with the text property

class HTMLTableParser:
    
    def parse_url(self, url):
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'lxml')
        return [(table['id'],self.parse_html_table(table))\
                for table in soup.find_all('table')]
        
    def parse_html_table(self, table):
        n_columns = 0
        n_rows=0
        column_names = []
        
        # Find number of rows and columns
        # we also find the column titles if we can
        for row in table.find_all('tr'):
            
            # Determine the number of rows in the table
            td_tags = row.find_all('td')
            if len(td_tags) > 0:
                n_rows+=1
                if n_columns == 0:
                     # Set the number of columns for our table
                     n_columns = len(td_tags)
            
            # Handle column names if we find them
            th_tags = row.find_all('th')
            if len(th_tags) > 0 and len(column_names) == 0:
                for th in th_tags:
                    column_names.append(th.get_text())
                    
        # Safeguard on Column Titles
        if len(column_names) > 0 and len(column_names) != n_columns:
            raise Exception("Column titles do not match the number of columns")
            
        columns = column_names if len(column_names) > 0 else range(0,n_columns)
         
        df = pd.DataFrame(columns = columns,index= range(0,n_rows))
         
        row_marker = 0
        for row in table.find_all('tr'):
             column_marker = 0
             columns = row.find_all('td')
             for column in columns:
                 df.iat[row_marker,column_marker] = column.get_text()
                 column_marker += 1
             if len(columns) > 0:
                 row_marker += 1
                 
        for col in df:
                try:
                     df[col] = df[col].astype(float)
                except ValueError:
                    pass
                
        return df
    
hp = HTMLTableParser()
df = hp.parse_url(url)[0][1]

In [3]:
df.info

<bound method DataFrame.info of         Case                                            Patient  \
0     6560.0                       37 Year Old Male Bangladeshi   
1     6558.0                       37 Year Old Male Bangladeshi   
2     6531.0                       31 Year Old Male Bangladeshi   
3     6529.0                            43 Year Old Male Indian   
4     6509.0                        31 Year Old Male Myanmarian   
5     6497.0                       30 Year Old Male Bangladeshi   
6     6489.0                       23 Year Old Male Bangladeshi   
7     6472.0                       26 Year Old Male Bangladeshi   
8     6471.0                       36 Year Old Male Bangladeshi   
9     6469.0                       28 Year Old Male Bangladeshi   
10    6453.0                       39 Year Old Male Bangladeshi   
11    6452.0                       41 Year Old Male Bangladeshi   
12    6449.0                            36 Year Old Male Indian   
13    6439.0                  

---
### 2. Inspecting the data
---


In [4]:
df.columns

Index(['Case', 'Patient', 'Age', 'Gender', 'Nationality', 'Status',
       'Infection Source', 'Country of Origin', 'Symptomatic ToConfirmation',
       'Days ToRecover', 'Symptomatic At', 'Confirmed At', 'Recovered At',
       'Displayed Symptoms?'],
      dtype='object')

In [5]:
df.isnull().sum()

Case                          0
Patient                       0
Age                           0
Gender                        0
Nationality                   0
Status                        0
Infection Source              0
Country of Origin             0
Symptomatic ToConfirmation    0
Days ToRecover                0
Symptomatic At                0
Confirmed At                  0
Recovered At                  0
Displayed Symptoms?           0
dtype: int64

In [6]:
df.head()

,Case,Patient,Age,Gender,Nationality,Status,Infection Source,Country of Origin,Symptomatic ToConfirmation,Days ToRecover,Symptomatic At,Confirmed At,Recovered At,Displayed Symptoms?
0,6560.0,37 Year Old Male Bangladeshi,\n 37\n ...,Male,Bangladeshi,In hospital,Local transmission,Unclear origin,-,-,-,"19th, Apr 2020",-,true
1,6558.0,37 Year Old Male Bangladeshi,\n 37\n ...,Male,Bangladeshi,In hospital,Local transmission,Unclear origin,-,-,-,"19th, Apr 2020",-,true
2,6531.0,31 Year Old Male Bangladeshi,\n 31\n ...,Male,Bangladeshi,In hospital,Local transmission,Unclear origin,-,-,-,"19th, Apr 2020",-,true
3,6529.0,43 Year Old Male Indian,\n 43\n ...,Male,Indian,In hospital,Local transmission,Unclear origin,-,-,-,"19th, Apr 2020",-,true
4,6509.0,31 Year Old Male Myanmarian,\n 31\n ...,Male,Myanmarian,In hospital,Local transmission,Unclear origin,-,-,-,"18th, Apr 2020",-,true


In [7]:
age = df.Age

df.Age = df.Age.replace(r'\n',  '', regex=True)
df.Age = df.Age.replace(r'\ ',  '', regex=True)
df.Age = df.Age.replace(r'',  '1', regex=True)
df.Age = df.Age.replace(r'-',  '1', regex=True)


In [8]:
df.Age = pd.to_numeric(df.Age)
df.Age.isnull()
if df.Age.isnull == 0:
    print("mother me")

In [9]:
bins = [0, 10, 20, 30, 40, 50, 60, 70, 80, 120]
# bins = [9, 19, 29, 39, 49, 59, 69, 79, 89, 120]

labels = ['1-9', '10-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70-79', '>80']
df['Agegroup'] = pd.cut(df.Age, bins = bins, right=False, labels = labels, include_lowest = True)

df.head(100)

,Case,Patient,Age,Gender,Nationality,Status,Infection Source,Country of Origin,Symptomatic ToConfirmation,Days ToRecover,Symptomatic At,Confirmed At,Recovered At,Displayed Symptoms?,Agegroup
0,6560.0,37 Year Old Male Bangladeshi,37,Male,Bangladeshi,In hospital,Local transmission,Unclear origin,-,-,-,"19th, Apr 2020",-,true,30-39
1,6558.0,37 Year Old Male Bangladeshi,37,Male,Bangladeshi,In hospital,Local transmission,Unclear origin,-,-,-,"19th, Apr 2020",-,true,30-39
2,6531.0,31 Year Old Male Bangladeshi,31,Male,Bangladeshi,In hospital,Local transmission,Unclear origin,-,-,-,"19th, Apr 2020",-,true,30-39
3,6529.0,43 Year Old Male Indian,43,Male,Indian,In hospital,Local transmission,Unclear origin,-,-,-,"19th, Apr 2020",-,true,40-49
4,6509.0,31 Year Old Male Myanmarian,31,Male,Myanmarian,In hospital,Local transmission,Unclear origin,-,-,-,"18th, Apr 2020",-,true,30-39
5,6497.0,30 Year Old Male Bangladeshi,30,Male,Bangladeshi,In hospital,Local transmission,Unclear origin,-,-,-,"18th, Apr 2020",-,true,30-39
6,6489.0,23 Year Old Male Bangladeshi,23,Male,Bangladeshi,In hospital,Local transmission,Unclear origin,-,-,-,"18th, Apr 2020",-,true,20-29
7,6472.0,26 Year Old Male Bangladeshi,26,Male,Bangladeshi,In hospital,Local transmission,Unclear origin,-,-,-,"18th, Apr 2020",-,true,20-29
8,6471.0,36 Year Old Male Bangladeshi,36,Male,Bangladeshi,In hospital,Local transmission,Unclear origin,-,-,-,"18th, Apr 2020",-,true,30-39
9,6469.0,28 Year Old Male Bangladeshi,28,Male,Bangladeshi,In hospital,Local transmission,Unclear origin,-,-,-,"18th, Apr 2020",-,true,20-29


In [10]:
# Map lower case function and replace spaces with '_' to all column names for my own ease 
df.columns = map(str.lower, df.columns)
df.columns = df.columns.str.replace(" ", "_")

In [21]:
age_details_df = pd.DataFrame(df.agegroup.value_counts())
age_details_df.reset_index(inplace=True)
age_details_df["group"] = age_details_df['index']
age_details_df.rename(columns={"agegroup": "count"})
age_details_df.drop(['index'], axis=1, inplace=True)

age_details_df

,agegroup,group
0,2563,30-39
1,1989,20-29
2,1139,40-49
3,455,50-59
4,211,60-69
5,71,70-79
6,63,1-9
7,59,10-19
8,38,>80


In [22]:
# check out posts that are duplicates based on case number
df[df['case'].duplicated(keep=False)]

,case,patient,age,gender,nationality,status,infection_source,country_of_origin,symptomatic_toconfirmation,days_torecover,symptomatic_at,confirmed_at,recovered_at,displayed_symptoms,agegroup


In [23]:
# Remove question mark in column name
df = df.rename(columns={'displayed_symptoms?': 'displayed_symptoms'})

In [24]:
df.symptomatic_toconfirmation.value_counts()

-     6343
1       31
2       29
5       28
4       22
3       22
6       19
9       18
8       16
7       15
10      11
12       7
15       5
14       4
13       3
16       3
0        3
11       3
17       2
19       1
22       1
33       1
31       1
Name: symptomatic_toconfirmation, dtype: int64

In [25]:
# save combined data in new DataFrame
df.to_csv('./datasets/df_sg_covid.csv', index=False)
age_details_df.to_csv('./datasets/age_details.csv', index=False)